# Lab 3: Linearisation and phase portraits

By the end of this Lab, we will be using Python to produce phase portraits for different $2\times2$ systems of ODEs. This will be a good opportunity for you to review theory from the lectures, and to revisit some of the tutorial problems.

Before producing phase portraits, we will recap some of the theory from earlier in the course, about producing solutions for systems of ODEs defined by a matrix equation, and about linearising nonlinear equations.

## 1. Matrix solutions 

To get started, we will revisit Q1 from the Week 3 tutorial sheet, and use Python to find the general solution for the system. (You may also want to open the model solutions for the tutorial sheet, to follow the method.) 

***
### Example
Find the general solution of the system of ODEs given by the matrix equation:

$$\mathbf{x}'=\begin{bmatrix}3 & -4 \\ 1 & -1\end{bmatrix}\mathbf{x}.$$
***

Recall from Lab 1 that we can use SymPy to find the eigenvalues and eigenvectors:

In [ ]:
import sympy as sym
sym.init_printing()
from IPython.display import display_latex

M = sym.Matrix([[3,-4],[1,-1]])
evs = M.eigenvects()
evs

This shows that $\lambda=1$ is the only eigenvalue, with algebraic multiplicity $2$, and corresponding eigenspace spanned by the eigenvector $\begin{bmatrix}2\\1\end{bmatrix}$.

For reference, note that we can also use `numpy` to do this numerically:

In [ ]:
import numpy as np
from numpy import linalg as LA
w, v = LA.eig(np.array([[3,-4], [1,-1]]))
w, v

<span class="label label-danger">Task</span>
The two approaches seem to give quite different-looking answers. Do you understand how they actually agree? Can you reason why numpy picked these values (what's special about them)?

YOUR ANSWER HERE

Returning to our symbolic solution, let's extract the eigenvector:

In [ ]:
xi1 = evs[0][2][0]
xi1

<span class="label label-danger">Task</span>
In a similar way, extract the eigenvalue and store it in a variable called `e_val`.

In [ ]:
# your code here

# YOUR CODE HERE
raise NotImplementedError()

We can then used the `sympy` function `linsolve` to solve the system $(A-\lambda\mathbb{I})\mathbf{\eta} = \mathbf{\xi}_1$:

In [ ]:
eta1, eta2 = sym.symbols('eta1, eta2')
eta_sol = sym.linsolve((M-e_val*sym.eye(2), xi1), eta1, eta2)
eta_sol

In general, `linsolve((A,b), x, y)` will solve the system $A\begin{bmatrix}x\\y\end{bmatrix}=\mathbf{b}$ (you can see more in [the documentation](https://docs.sympy.org/latest/modules/solvers/solveset.html#linsolve)).

The solution we get here is a general solution with a free parameter, so we can simply choose one by substituting $\eta_2=0$. Let's do that and then turn the result into a vector:

In [ ]:
eta_soln = list(eta_sol)[0].subs(eta2, 0)
eta_vect = sym.Matrix(eta_soln)
eta_vect

Now we're ready to build the general solution, which will be in the form $$C_1\mathbf{\xi}_1e^{r_1t} + C_2\left(\mathbf{\xi}_1 te^{r_1t}+\mathbf{\eta}e^{r_1t}\right),$$
where $r_1$ was our eigenvalue.

In our notation so far, this is:

In [ ]:
C1, C2, t = sym.symbols('C1, C2, t')
gen_soln = C1*xi1*sym.exp(e_val*t) + C2*(xi1*t*sym.exp(e_val*t) + eta_vect*sym.exp(e_val*t))
gen_soln

This doesn't look quite like we'd expect. This is because SymPy automatically adds everything together in order to simplify, rather than keeping the terms separate as we would tend to do when writing it by hand.

There are [a couple of ways](https://docs.sympy.org/latest/tutorial/manipulation.html#prevent-expression-evaluation) to force SymPy to keep the terms separate. The first is to use explicit `MatAdd` and `MatMul` commands to do the additions and multiplications, with `evaluate = False` to keep SymPy from simplifying:

In [ ]:
gen_sol = sym.MatAdd(sym.MatMul(C1, xi1, sym.exp(e_val*t)),
           sym.MatMul(C2,
                      sym.MatAdd(sym.MatMul(xi1,t,sym.exp(e_val*t)),
                                sym.MatMul(eta_vect, sym.exp(e_val*t)))), evaluate = False)
gen_sol

The second way is to use `UnevaluatedExpr` to keep the vectors from being evaluated (i.e. simplified):

In [ ]:
from sympy import UnevaluatedExpr
xi1_vect = UnevaluatedExpr(xi1)
gen_sol2 = C1*xi1_vect*sym.exp(e_val*t) + C2*(xi1_vect*t*sym.exp(e_val*t) + UnevaluatedExpr(eta_vect)*sym.exp(e_val*t))
gen_sol2

<div class="alert alert-info">
    <h3>Exercise 1.1</h3>

**(a)** Create a function `pretty_solution` which carries out the process above for any given system. It should take in a single Matrix (which you can assume is $2\times2$ and has a single repeated eigenvalue) and return the corresponding general solution. 

**(b)** Use your function to find the general solutions for these problems, some of which appear in Section 7.8 of Boyce & DiPrima:

1. $\mathbf{x}'=\begin{bmatrix}3 & 1 \\ -4 & -1\end{bmatrix}\mathbf{x}$,
2. $\mathbf{x}'=\begin{bmatrix}4 & 8 \\ -2 & -4\end{bmatrix}\mathbf{x}$,
3. $\mathbf{x}'=\begin{bmatrix}-\frac32 & -\frac14 \\ 1 & -\frac12\end{bmatrix}\mathbf{x}$,
4. $\mathbf{x}'=\begin{bmatrix}-3 & -\frac52 \\ \frac52 & 2\end{bmatrix}\mathbf{x}$.

    
</div>

_Tip:_ For 3 and 4, you will find that if you just type the fractions then SymPy will return solutions which have decimals in them. To make sure the calculations are done exactly, you should enter the fractions using the [Rational](https://docs.sympy.org/latest/modules/core.html#rational) function.

_Note:_ You can check your answers against those given in the back of Boyce & DiPrima, and also against the result of applying SymPy's `dsolve` function.

In [ ]:
# Your code here for pretty_solution()

def pretty_solution(M):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# Autograder test
def check_solution(student_answer, expected_answer, message):
    print("Testing "+message)
    # Check the constants
    const = list(student_answer.atoms(sym.Symbol))
    assert t in const
    assert len(const) == 3
    print(" ✔ output contains the correct number of symbols")
    assert isinstance(student_answer,type(gen_sol))
    print(" ✔ output is of the correct type")
    # Student answer should use C1 and C2 as the constants, but here we replace them just in case
    const.remove(t)
    test1 = student_answer.subs([(const[0], C1),(const[1], C2)])
    test2 = student_answer.subs([(const[1], C1),(const[0], C2)])
    # Does the answer match the model answer?
    assert test1 - expected_answer == sym.Matrix(2,1,[0,0]) or test2 - expected_answer == sym.Matrix(2,1,[0,0])
    print(" ✔ general solution matches model answer")

check_solution(pretty_solution(sym.Matrix(2,2,[3,-4,1,-1])), gen_sol, "gen_sol example")

In [ ]:
# Your code here for problem 1

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Your code here for problem 2

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Your code here for problem 3

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Your code here for problem 4

# YOUR CODE HERE
raise NotImplementedError()

## 2. Critical points and linearisation

Another important step towards producing phase portraits is to identify the critical point(s) of the system, and to consider the behaviour of the linearised system around those points.

In this section of the Lab, we will use SymPy to do some of this work.

***
### Example
As in Examples 2-3 of Boyce & DiPrima Section 9.3, the motion of a pendulum is described by the system $$\frac{dx}{dt}=y, \quad \frac{dy}{dt}=-\omega^2 \sin x - \gamma y.$$
We will use SymPy to find the linear approximation of the system about the critical points $(0,0)$ and $(\pi,0)$.
***

First, we create expressions to represent the equations. Note that we use the names `Fxy` and `Gxy` to echo the notation used in equation 15 in Boyce & DiPrima.

In [ ]:
x = sym.Function('x')
y = sym.Function('y')
omega, gamma = sym.symbols('omega, gamma')
Fxy = y(t)
Gxy = -omega**2*sym.sin(x(t))-gamma*y(t)

To compute the linearisation, we can find the Jacobian $\begin{pmatrix}F_x & F_y \\ G_x & G_y\end{pmatrix}$. This can be done in SymPy using the [`jacobian` function](https://docs.sympy.org/0.6.7/modules/matrices.html#sympy.matrices.matrices.Matrix.jacobian) -- to use this, we first put our two functions into a SymPy Matrix; then we can apply the `jacobian` function and substitute the values from our critical points.

In [ ]:
FGmat = sym.Matrix([Fxy,Gxy])
FGmat

In [ ]:
matJ = FGmat.jacobian([x(t), y(t)])
matJ

In [ ]:
matJatCP = matJ.subs(x(t),0).subs(y(t),0)
matJatCP

We can then form the linear system (like equation 17 in Boyce & DiPrima):

In [ ]:
linsys_atCP = sym.Eq(sym.Matrix([x(t), y(t)]).diff(t), sym.MatMul(matJatCP, sym.Matrix([x(t), y(t)])))
linsys_atCP

Similarly, we can find the linearisation around $(\pi,0)$:

In [ ]:
matJatCP2 = matJ.subs(x(t),sym.pi).subs(y(t),0)
matJatCP2

In [ ]:
u = sym.Function('u')
v = sym.Function('v')
linsys_atCP2 = sym.Eq(sym.Matrix([u(t), v(t)]).diff(t), sym.MatMul(matJatCP2, sym.Matrix([u(t), v(t)])))
linsys_atCP2

Note that this time, the equation is expressed in terms of $u=x-\pi$ and $v=y$ -- recall that, in general, the linearised system about $(x_0,y_0)$ is given by $$\frac{d\mathbf{u}}{dt}=\mathbf{J}(x_0,y_0)\mathbf{u},$$ where $\mathbf{J}(x_0,y_0)$ is the Jacobian matrix evaluated at $(x_0,y_0)$ and $\mathbf{u}=\begin{pmatrix}u_1\\u_2\end{pmatrix}=\begin{pmatrix}x-x_0\\y-y_0\end{pmatrix}$

***
### Example

We can use a similar approach to solve Tutorial 5, Q1i: finding the linearisation of $$x'=(2+x)(y-x),\quad y'=(4-x)(y+x)$$ about the critical points.
***

We start by defining the equations:

In [ ]:
x = sym.Function('x')
y = sym.Function('y')
eq1 = sym.Eq(x(t).diff(t),(2+x(t))*(y(t)-x(t)))
eq2 = sym.Eq(y(t).diff(t),(4-x(t))*(y(t)+x(t)))
[eq1, eq2]

Like before, we form a Matrix ready to compute the Jacobian:

In [ ]:
FG = sym.Matrix([eq1.rhs, eq2.rhs])
FG

But first, we can use SymPy's `solve` function to find the critical points -- here, we find where `FG` is zero, which is equivalent to solving $x'=0, y'=0$:

In [ ]:
sym.solve(FG)

Now we compute the Jacobian:

In [ ]:
matJ = FG.jacobian([x(t), y(t)])
matJ

The linearisation about the critical point $(0,0)$ is then:

In [ ]:
lin_mat = matJ.subs({x(t):0, y(t):0})
lin_mat

In [ ]:
lin_mat * sym.Matrix([u(t),v(t)])

Note that we can also use SymPy to help us determine the behaviour around this critical point, by computing the eigenvalues and eigenvectors:

In [ ]:
lin_mat.eigenvects()

If we just want the eigenvalues, we can extract them more simply as:

In [ ]:
evals = list(lin_mat.eigenvals().keys())
evals

In [ ]:
list(map(sym.N, evals))

<span class="label label-danger">Task</span>
What is going on in this final line of code?

<div class="alert alert-info">
    <h3>Exercise 2.1 </h3>

Find the linearisation about each of the other critical points, and compute the eigenvalues for the corresponding linear systems.

</div>

In [ ]:
# Your code here for critical point (-2,2)
# Use the variable name lin_cp2 for the linearisation
# Store the egeinvalues in a list evals_cp2

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Autograded tests

assert isinstance(lin_cp2,type(matJ))
assert list(lin_cp2) == [4,0,6,6]
print("✔ Your matrix has the correct entries")

assert isinstance(evals_cp2,list)
assert len(evals_cp2) == 2
print("✔ You have the correct number of eigenvalues")

assert lin_cp2 == matJ.subs({x(t):-2, y(t):2})
print("✔ The linearisation is correct")
s_evals_cp2 = list(lin_cp2.eigenvals().keys())
p_evals_cp2 = list(matJ.subs({x(t):-2, y(t):2}).eigenvals().keys())
assert evals_cp2 == s_evals_cp2
print("✔ The eigenvalues are correctly calculated")
assert evals_cp2 == p_evals_cp2
print("✔ Your solution agrees with the model answer")

In [ ]:
# Your code here for critical point (4,4)
# Use the variable name lin_cp3 for the linearisation
# Store the critical points in a list evals_cp3

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Autograded tests

assert isinstance(lin_cp3,type(matJ))
assert list(lin_cp3) == [-6,6,-8,0]
print("✔ Your matrix has the correct entries")

assert isinstance(evals_cp3,list)
assert len(evals_cp3) == 2
print("✔ You have the correct number of eigenvalues")

assert lin_cp3 == matJ.subs({x(t):4, y(t):4})
print("✔ The linearisation is correct")
s_evals_cp3 = list(lin_cp3.eigenvals().keys())
p_evals_cp3 = list(matJ.subs({x(t):4, y(t):4}).eigenvals().keys())
assert evals_cp3 == s_evals_cp3
print("✔ The eigenvalues are correctly calculated")
assert evals_cp3 == p_evals_cp3
print("✔ Your solution agrees with the model answer")

<div class="alert alert-info">
    <h3>Exercise 2.2</h3>

**(a)** Create two functions to carry out this process:

* `lin_matrix` which takes in a system of two equations (given as a list of SymPy Equations) and the coordinates of a point (given as a list), and returns the Jacobian matrix of the system evaluated at that point. 

* `linearise` which takes in the same arguments, and returns a system of linear equations in $u$ and $v$. 

**(b)** Use your functions to solve problem 11 from section 9.3 of Boyce & DiPrima:

Find all the critical points of the system $$x'=x-x^2-xy,\quad y'=3y-xy-2y^2$$ and find the corresponding linear system near each critical point.

</div>

As an example, your functions in (a) should produce the following output using the equations defined in the previous example:

`lin_matrix([eq1,eq2],[0,0])`

$\displaystyle \left[\begin{matrix}-2 & 2\\4 & 4\end{matrix}\right]$

`linearise([eq1,eq2],[0,0])`

$\displaystyle \left[ \frac{d}{d t} u{\left(t \right)} = - 2 u{\left(t \right)} + 2 v{\left(t \right)}, \  \frac{d}{d t} v{\left(t \right)} = 4 u{\left(t \right)} + 4 v{\left(t \right)}\right]$


In [ ]:
# For part (a)
# Create the function lin_matrix

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Autograder test 

from inspect import signature
function_signature = signature(eval("lin_matrix"))
assert len(function_signature.parameters) == 2
print("✔ lin_matrix has the correct number of arguments")

assert lin_matrix([eq1,eq2],[0,0]) == sym.Matrix(2,2,[-2,2,4,4])
print("✔ Your solution produces the correct output for eq1 and eq2")

In [ ]:
# For part (a)
# Create the function linearise

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Autograder test 

from inspect import signature
function_signature = signature(eval("linearise"))
assert len(function_signature.parameters) == 2
print("✔ lin_matrix has the correct number of arguments")

assert linearise([eq1,eq2],[0,0]) == [sym.Eq(u(t).diff(t),-2*u(t)+2*v(t)),
                                     sym.Eq(v(t).diff(t),4*u(t)+4*v(t))]
print("✔ Your solution produces the correct output for eq1 and eq2")

In [ ]:
# For part (b)
# Enter the system of equations and display it as a list

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Autograder test 

assert (_ == [sym.Eq(x(t).diff(t), x(t)-x(t)**2-x(t)*y(t)),
              sym.Eq(y(t).diff(t), 3*y(t)-x(t)*y(t)-2*y(t)**2)] 
        or _ == [sym.Eq(y(t).diff(t), 3*y(t)-x(t)*y(t)-2*y(t)**2),
                 sym.Eq(x(t).diff(t), x(t)-x(t)**2-x(t)*y(t))])
print("✔ You correctly displayed the system ")

In [ ]:
# For part (b)
# Find the critical points and display them
# Use the variable name CPs for the array of critical points

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Autograder test
your_output = _
type(your_output) == list
len(your_output) == 4
print("✔ You displayed a list of the correct length")


In [ ]:
# For part (b)
# Find the linearisations and show the details

# YOUR CODE HERE
raise NotImplementedError()

## 3. Phase portraits

Plottng phase portraits can help to understand the behaviour of a nonlinear system, by showing what is happening to solution trajectories in the phase space and how this links to the linearised behaviour around the critical points.

We will base our approach on the sample code that Jacques provided in the `vectorFields.ipynb` notebook posted on Learn -- the following code comes from there, and shows a phase portrait for the pendulum (which was the first example in section 2, here with $\gamma=0, \omega=1$).

In [ ]:
# Import packages

import numpy as np
from matplotlib import pyplot as plt
from scipy.integrate import odeint
%matplotlib inline

# Define vector field

def vField(x,t):
    u = x[1]
    v = -np.sin(x[0])
    return [u,v]

# Plot vector field

X, Y = np.mgrid[-np.pi:np.pi:10j,-7:7:10j]
U, V = vField([X,Y],0)

fig, ax = plt.subplots(figsize=(10, 7))
ax.quiver(X, Y, U, V)

# Compute and plot 4 trajectories

t=np.linspace(0,2,100)
x0, y0 = -3,4
x = odeint(vField,[x0,y0],t)
ax.plot(x[:,0],x[:,1]);

t=np.linspace(0,8,100)
x0, y0 = 0,1
x = odeint(vField,[x0,y0],t)
ax.plot(x[:,0],x[:,1]);

t=np.linspace(0,12,100)
x0, y0 = -2,1
x = odeint(vField,[x0,y0],t)
ax.plot(x[:,0],x[:,1]);

t=np.linspace(0,2,100)
x0, y0 = 3,-4
x = odeint(vField,[x0,y0],t)
ax.plot(x[:,0],x[:,1]);

# 

plt.xlabel('x')
plt.ylabel('y')
plt.xlim(-3,3)

plt.show()

Here we make several changes to the plot:

* increasing the plot area and using a finer grid,
* setting the colours using [colormaps](https://matplotlib.org/examples/color/colormaps_reference.html),
* changing some settings on the `quiver` which is used to produce the direction field, e.g. making the arrows centred at the points of the grid,
* using a loop to draw the trajectories,
* adding a few more trajectories to highlight features,
* adding points (in blue) to show where the trajectories started,
* adding the legend to the plot.

<span class="label label-danger">Task</span>
Make sure you can see all of these changes in the code. Play around with them by adjusting the various settings, e.g. make the grid more or less fine, change the scale of the arrows in the direction field, etc.

In [ ]:
# Define vector field
def vField(x,t):
    u = x[1]
    v = -np.sin(x[0])
    return [u,v]

# Plot vector field
X, Y = np.mgrid[-2*np.pi:2*np.pi:30j,-7:7:20j]
U, V = vField([X,Y],0)

# define colours for each vector based on their lengths
M = np.hypot(U, V)

fig, ax = plt.subplots(figsize=(10, 7))
ax.quiver(X, Y, U, V, M, scale=1/0.01, pivot = 'mid', cmap = plt.cm.bone)

# Settings for trajectories
ics  = [[-6,4], [0,1], [-2,1], [6,-4], [-4,2], [-4, 1], [4, -1], [-5, 1.5], [5, -1.5]]
durations = [[0,4], [0,8], [0,12], [0,4], [0,4], [0,8], [0,8], [0,4], [0,4]]
vcolors = plt.cm.autumn_r(np.linspace(0.5, 1., len(ics)))  # colors for each trajectory

# plot trajectories
for i, ic in enumerate(ics):
    t = np.linspace(durations[i][0], durations[i][1],100)
    x = odeint(vField, ic, t)
    ax.plot(x[:,0], x[:,1], color=vcolors[i], label='X0=(%.f, %.f)' % (ic[0], ic[1]) )

ic_x = [ic[0] for ic in ics]
ic_y = [ic[1] for ic in ics]
ax.scatter(ic_x, ic_y, color='blue', s=20)
# note: you can replace ic_x, ic_y with *list(zip(*ics)) but this is a bit cryptic!

plt.xlabel('x')
plt.ylabel('y')
plt.xlim(-6,6)
plt.legend()

plt.show()

<div class="alert alert-info">
    <h3>Exercise 3.1</h3>

Produce phase portraits for the following systems in Tutorial 5 Q2:

**(i)** $\mathbf{x}'=\begin{pmatrix}3 & -2\\2 & -2\end{pmatrix}\mathbf{x}$

**(iii)** $\mathbf{x}'=\begin{pmatrix}1 & -4\\4 & -7\end{pmatrix}\mathbf{x}$

</div>

_Note:_ Choosing good starting points for the trajectories is a bit of an art; you will need to adjust the values that you pick to find the best ones. You might also find that plotting the trajectories over a duration which is too long will cause the plot to break -- either you will get an error message saying the execution took too long, or you will find that the trajectory has shot well beyond the grid of arrows -- so you should adjust these values carefully too.

In [ ]:
# your code here for part (i)

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# your code here for part (iii)

# YOUR CODE HERE
raise NotImplementedError()

<div class="alert alert-info">
    <h3>Exercise 3.2</h3>

Produce a phase portrait for the system we analysed in Exercise 2.2(b). Make sure you show the features around all of the critical points.

</div>


In [ ]:
# your code here

# YOUR CODE HERE
raise NotImplementedError()